In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
data=Path("Resources/lending_data.csv")
df_lending_data = pd.read_csv(data)

# Review the DataFrame
df_lending_data.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [3]:
# Separate the data into labels and features

# Separate the y variable, the labels
y= df_lending_data["loan_status"]

# Separate the X variable, the features
X = df_lending_data.drop(columns="loan_status")

In [4]:
# Review the y variable Series
y

0        0
1        0
2        0
3        0
4        0
        ..
77531    1
77532    1
77533    1
77534    1
77535    1
Name: loan_status, Length: 77536, dtype: int64

In [5]:
# Review the X variable DataFrame
X

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000
...,...,...,...,...,...,...,...
77531,19100.0,11.261,86600,0.653580,12,2,56600
77532,17700.0,10.662,80900,0.629172,11,2,50900
77533,17600.0,10.595,80300,0.626401,11,2,50300
77534,16300.0,10.068,75300,0.601594,10,2,45300


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [6]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [7]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [8]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='liblinear',max_iter=200,random_state=1)

# Fit the model using training data
classifier.fit(X_train,y_train)

LogisticRegression(max_iter=200, random_state=1, solver='liblinear')

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [9]:
# Make a prediction using the testing data
y_predictions = classifier.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [19]:
# Print the balanced_accuracy score of the model
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

balanced_accuracy = balanced_accuracy_score(y_test, y_predictions)
balanced_accuracy

Training Data Score: 0.9920896959691842
Testing Data Score: 0.9919521254643004


0.9544445434497851

In [11]:
# Generate a confusion matrix for the model
model_confusion_matrix = confusion_matrix(y_test, y_predictions)
model_confusion_matrix

array([[18662,   103],
       [   53,   566]], dtype=int64)

In [15]:
# Print the classification report for the model
model_classification_report = classification_report(y_test,y_predictions,target_names=["Healthy Loans","High-Risk Loans"] )
print(model_classification_report)

                 precision    recall  f1-score   support

  Healthy Loans       1.00      0.99      1.00     18765
High-Risk Loans       0.85      0.91      0.88       619

       accuracy                           0.99     19384
      macro avg       0.92      0.95      0.94     19384
   weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model has an overall accuracy of 99% and a balanced accuracy score of 95.44%. For Healthy loans (0) the model precision is at 100% while the recall is 99% with a 100% f1 score. However, the model does not perform as well for the High-Risk loans (1) predictions, the model precision drops to 85% and the f1 score to 88%. The recall is 99% and 91% for the healthy and high-risk bale predictions respectively. Type I or false postives are 0.55% and Type II or false negatives are 8.56% respectively.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [12]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
ros = RandomOverSampler(random_state=1)


# Fit the original training data to the random_oversampler model
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

In [13]:
# Count the distinct values of the resampled labels data
y_train_resampled.value_counts()

0    56271
1    56271
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [14]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
resampled_model_classifier = LogisticRegression(solver='liblinear',max_iter=200,random_state=1)


# Fit the model using the resampled training data
resampled_model_classifier.fit(X_train_resampled,y_train_resampled)


# Make a prediction using the testing data
y_resampled_predictions = resampled_model_classifier.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [20]:
# Print the balanced_accuracy score of the model 
print(f"Training Data Score: {resampled_model_classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {resampled_model_classifier.score(X_test, y_test)}")
balanced_accuracy_resampled = balanced_accuracy_score(y_test,y_resampled_predictions)
balanced_accuracy_resampled

Training Data Score: 0.9945831613702022
Testing Data Score: 0.9938093272802311


0.9936781215845847

In [16]:
# Generate a confusion matrix for the model
resampled_model_confusion_matrix = confusion_matrix(y_test, y_resampled_predictions)
resampled_model_confusion_matrix

array([[18649,   116],
       [    4,   615]], dtype=int64)

In [17]:
# Print the classification report for the model
resampled_model_classification_report = classification_report(y_test,y_resampled_predictions,target_names=["Healthy Loans","High-Risk Loans"] )
print(resampled_model_classification_report)

                 precision    recall  f1-score   support

  Healthy Loans       1.00      0.99      1.00     18765
High-Risk Loans       0.84      0.99      0.91       619

       accuracy                           0.99     19384
      macro avg       0.92      0.99      0.95     19384
   weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The overall accuracy of the model with the oversampled data remains the same at 99%. The balanced accuracy score increased from 95% to 99% after resampling. The mean accuracy for the resampled training data and the hold out test data are at 99% suggesting the prediction results are good. The precision for healthy loans (0) is 100% and for high-risk loans (1) it drops to 84%. The type 1 error or false positives is 0.62% and type 2 error or false negatives is 0.65%.